In [ ]:
pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain-openai in c:\users\hp\downloads\lib\site-packages (0.1.17)




[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install --upgrade --quiet  langchain-experimental

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import uuid
import os
import base64
import pdfplumber
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
from langchain.schema import Document
from typing import List, Tuple
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI


c:\Users\Hp\Downloads\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Function to extract text from a PDF file using pdfplumber
def extract_pdf_text(file_path):
    print(f"Extracting text from {file_path}...")
    with pdfplumber.open(file_path) as pdf:
        text_by_page = []
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                text_by_page.append(text)
            else:
                print(f"Warning: No text extracted from page {page.page_number}.")
    print(f"Extracted text from {len(text_by_page)} pages.")
    return text_by_page

# Function to extract images from a PDF file using pdfplumber and save them
def extract_pdf_images(file_path, output_folder):
    print(f"Extracting images from {file_path}...")
    extracted_images = []
    os.makedirs(output_folder, exist_ok=True)

    with pdfplumber.open(file_path) as pdf:
        for page_number, page in enumerate(pdf.pages):
            images = page.images
            for img_index, img in enumerate(images):
                bbox = (max(img["x0"], 0), max(img["top"], 0), min(img["x1"], page.width), min(img["bottom"], page.height))
                try:
                    image = page.within_bbox(bbox).to_image()
                    image_path = os.path.join(output_folder, f"page_{page_number + 1}_image_{img_index + 1}.png")
                    image.save(image_path)
                    extracted_images.append((image_path, page_number + 1))
                except ValueError as e:
                    print(f"Error extracting image from page {page_number + 1}, image {img_index + 1}: {e}")
                except Exception as e:
                    print(f"Unknown error extracting image from page {page_number + 1}, image {img_index + 1}: {e}")

    print(f"Extracted {len(extracted_images)} images.")
    return extracted_images

from sentence_transformers import SentenceTransformer, util
import numpy as np

# Function to summarize text elements with overlapping chunks
def summarize_text_with_transformers(text, model_name='all-MiniLM-L6-v2', chunk_size=500, overlap=400):
    print("Summarizing text with overlap...")
    text_length = len(text)
    summaries = []

    # Load the sentence transformer model
    model = SentenceTransformer(model_name)

    for i in range(0, text_length, chunk_size - overlap):
        chunk = text[i:i + chunk_size]
        print(f"Processing text chunk {i // (chunk_size - overlap) + 1}/{(text_length // (chunk_size - overlap)) + 1}...")

        # Split chunk into sentences
        sentences = chunk.split('. ')

        if len(sentences) == 0:
            continue

        # Encode sentences
        embeddings = model.encode(sentences, convert_to_tensor=True)

        # Calculate cosine similarity matrix
        cosine_sim_matrix = util.cos_sim(embeddings, embeddings)

        # Convert tensor to NumPy array
        cosine_sim_matrix_np = cosine_sim_matrix.cpu().numpy()

        # Calculate the average similarity of each sentence to all others
        avg_cosine_sim = np.mean(cosine_sim_matrix_np, axis=1)

        # Find the most representative sentence
        most_representative_idx = np.argmax(avg_cosine_sim)
        summary_sentence = sentences[most_representative_idx]

        summaries.append(summary_sentence)

    print("Text summarized.")
    return " ".join(summaries)



# Function to summarize image elements using BLIP
def summarize_image(image_path, blip_model_name='Salesforce/blip-image-captioning-base'):
    print(f"Summarizing image {image_path} using BLIP...")

    # Load the BLIP processor and model
    processor = BlipProcessor.from_pretrained(blip_model_name)
    model = BlipForConditionalGeneration.from_pretrained(blip_model_name)

    # Open the image
    image = Image.open(image_path).convert("RGB")

    # Preprocess the image
    inputs = processor(images=image, return_tensors="pt")

    # Generate the summary (caption)
    outputs = model.generate(**inputs)
    summary = processor.decode(outputs[0], skip_special_tokens=True)

    print("Image summary completed.")
    return summary

# Function to create documents
def create_documents(text_chunks, image_paths, text_summaries, image_summaries, file_paths):
    print("Creating documents...")
    documents = []
    errors = []

    try:
        for idx, (text, summary) in enumerate(zip(text_chunks, text_summaries)):
            try:
                doc_id = str(uuid.uuid4())
                doc = Document(
                    page_content=summary,
                    metadata={
                        'id': doc_id,
                        'type': 'text',
                        'original_content': text,
                        'source': file_paths[idx % len(file_paths)]
                    }
                )
                documents.append(doc)
                print(f"Created document for text {idx + 1}: {doc_id}")
            except Exception as e:
                error_msg = f"Error creating document for text {idx + 1}: {e}"
                print(error_msg)
                errors.append(error_msg)

        for idx, (image_path, page_number) in enumerate(image_paths):
            try:
                doc_id = str(uuid.uuid4())
                doc = Document(
                    page_content=str(image_summaries[idx]),
                    metadata={
                        'id': doc_id,
                        'type': 'image',
                        'original_content': os.path.basename(image_path),
                        'source': file_paths[idx % len(file_paths)],
                        'page_number': page_number
                    }
                )
                documents.append(doc)
                print(f"Created document for image {idx + 1}: {doc_id}")
            except Exception as e:
                error_msg = f"Error creating document for image {idx + 1}: {e}"
                print(error_msg)
                errors.append(error_msg)

        print(f"Created {len(documents)} documents.")
    except Exception as e:
        error_msg = f"Unknown error during document creation: {e}"
        print(error_msg)
        errors.append(error_msg)

    if errors:
        print("Errors encountered during document creation:")
        for error in errors:
            print(error)

    return documents


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

In [ ]:
# Example usage:
file_paths = [r"C:\Users\Hp\Downloads\vsrag\pdf1.pdf", r"C:\Users\Hp\Downloads\vsrag\pdf2.pdf"]
output_folder = './images'

# Assuming extract_pdf_text and extract_pdf_images are defined elsewhere
all_text_chunks = []
all_image_paths = []

for file_path in file_paths:
    text_by_page = extract_pdf_text(file_path)
    all_text_chunks.extend(text_by_page)

    images = extract_pdf_images(file_path, output_folder)
    all_image_paths.extend(images)

text_summaries = []
for idx, text in enumerate(all_text_chunks):
    print(f"Summarizing text chunk {idx + 1}/{len(all_text_chunks)}")
    summary = summarize_text_with_transformers(text)
    text_summaries.append(summary)

image_summaries = []
for idx, image_info in enumerate(all_image_paths):
    if isinstance(image_info, tuple):
        image_path, _ = image_info
    else:
        image_path = image_info

    print(f"Summarizing image {idx + 1}/{len(all_image_paths)}")
    summary = summarize_image(image_path)
    image_summaries.append(summary)

Extracting text from C:\Users\Hp\Downloads\vsrag\pdf1.pdf...
Extracted text from 46 pages.
Extracting images from C:\Users\Hp\Downloads\vsrag\pdf1.pdf...
Extracted 34 images.
Extracting text from C:\Users\Hp\Downloads\vsrag\pdf2.pdf...
Extracted text from 18 pages.
Extracting images from C:\Users\Hp\Downloads\vsrag\pdf2.pdf...
Extracted 7 images.
Summarizing text chunk 1/64
Summarizing text with overlap...
Processing text chunk 1/1...
Text summarized.
Summarizing text chunk 2/64
Summarizing text with overlap...
Processing text chunk 1/4...
Processing text chunk 2/4...
Processing text chunk 3/4...
Processing text chunk 4/4...
Text summarized.
Summarizing text chunk 3/64
Summarizing text with overlap...
Processing text chunk 1/1...
Text summarized.
Summarizing text chunk 4/64
Summarizing text with overlap...
Processing text chunk 1/30...
Processing text chunk 2/30...
Processing text chunk 3/30...
Processing text chunk 4/30...
Processing text chunk 5/30...
Processing text chunk 6/30...
Pr

c:\Users\Hp\Downloads\Lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Image summary completed.
Summarizing image 2/41
Summarizing image ./images\page_1_image_2.png using BLIP...
Image summary completed.
Summarizing image 3/41
Summarizing image ./images\page_3_image_1.png using BLIP...
Image summary completed.
Summarizing image 4/41
Summarizing image ./images\page_7_image_1.png using BLIP...
Image summary completed.
Summarizing image 5/41
Summarizing image ./images\page_17_image_1.png using BLIP...
Image summary completed.
Summarizing image 6/41
Summarizing image ./images\page_19_image_1.png using BLIP...
Image summary completed.
Summarizing image 7/41
Summarizing image ./images\page_20_image_1.png using BLIP...
Image summary completed.
Summarizing image 8/41
Summarizing image ./images\page_23_image_1.png using BLIP...
Image summary completed.
Summarizing image 9/41
Summarizing image ./images\page_27_image_1.png using BLIP...
Image summary completed.
Summarizing image 10/41
Summarizing image ./images\page_27_image_2.png using BLIP...
Image summary complet

In [ ]:
# Print text and image summaries
print("Text Summaries:")
for idx, summary in enumerate(text_summaries):
    print(f"Text {idx + 1} Summary: {summary}")

print("\nImage Summaries:")
for idx, summary in enumerate(image_summaries):
    print(f"Image {idx + 1} Summary: {summary}")

Text Summaries:
Text 1 Summary: Fashion Design (Short Course)
Unit 1
Fashion Design (Short Course)
P a ge 1|46
Text 2 Summary: Page:23 to32
TOPIC 3: Principles of Design……………………………………………………………………. Page:23 to32
TOPIC 3: Principles of Design……………………………………………………………………. Page:33to45
Assessment ………………………………………………………………………………… ……………………………………………………………………………
Text 3 Summary: Fashion Design (Short Course)
Topic 1
P a ge 3|46
Text 4 Summary: One of the most puzzling aspects of clothing is fashion.
As mentioned above, clothing often clearly r Different types of
clothing clearly reveal status, especially in the past but even today, and also the wearers’ personalities.
This need to conceal and also to reveal relies on a range of different clothing types and will often be
chosen by wearers to do both in the past but even today, and also the wearers’ personalities.
This need to conceal and also to reveal relies on a range of different clothing types and will often be
chosen by wearers to do both One of

In [ ]:
# Create documents
documents = create_documents(all_text_chunks, all_image_paths, text_summaries, image_summaries, file_paths)

# Print confirmation of documents created successfully
print(f"Documents created successfully: {len(documents)} documents.")


Creating documents...
Created document for text 1: 22947539-be72-400a-81eb-815a4691617e
Created document for text 2: e0beba41-57bb-4233-8b7d-d5d126d2f141
Created document for text 3: 86c9072e-ec8f-47a6-bf3d-ada822592165
Created document for text 4: ea6e2cfc-9599-4fea-a5c0-63f5b0f38e61
Created document for text 5: 38e6e5dc-5c38-4e84-905a-03adfce77bf7
Created document for text 6: 6fd3ba0f-5122-4f06-b205-3394fee0541e
Created document for text 7: 22e4b7c9-6fdf-4e48-b833-b57b6455d186
Created document for text 8: 4e941d8b-b5e2-4696-b861-9bec6251eefe
Created document for text 9: a100df47-df7b-40c7-b4da-05f40acf6b7f
Created document for text 10: 28e2a67f-4b8b-403c-9e59-395133396f64
Created document for text 11: f02fc413-9580-412c-8d57-5480d3896c7f
Created document for text 12: 9ffa8ff9-e7ec-4b28-9289-fa14648524cf
Created document for text 13: 94651c55-0606-45f7-ae8b-03bd386995b9
Created document for text 14: 3b4084f5-f139-4010-bda7-934c45a2a1d3
Created document for text 15: 71a1e342-834c-480a-

In [ ]:
print(documents[104].__dict__)


{'id': None, 'metadata': {'id': 'f7424e57-b53c-4b88-baaf-e4bbdcf4bd74', 'type': 'image', 'original_content': 'page_19_image_1.png', 'source': 'C:\\Users\\Hp\\Downloads\\vsrag\\pdf1.pdf', 'page_number': 19}, 'page_content': 'a collection of clothes and jackets', 'type': 'Document'}


In [ ]:
pip install open_clip_torch torch


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install faiss-cpu transformers langchain openai



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain.vectorstores import FAISS
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import os
import numpy as np
from langchain.schema import Document
import faiss

# Assume documents are a list of Document objects
# Separate text and image documents
text_documents = [doc for doc in documents if doc.metadata["type"] == "text"]
image_documents = [doc for doc in documents if doc.metadata["type"] == "image"]

# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define the base directory where the images are stored
image_base_directory = "C:\\Users\\Hp\\Downloads\\images"

def get_clip_embeddings(documents, model, processor):
    embeddings = []
    for doc in documents:
        if doc.metadata["type"] == "text":
            inputs = processor(text=doc.page_content, return_tensors="pt", padding=True, truncation=True)
        else:
            # Load the image from the file path specified in original_content
            image_path = os.path.join(image_base_directory, doc.metadata["original_content"])
            image = Image.open(image_path)
            inputs = processor(images=image, return_tensors="pt")

        with torch.no_grad():
            if doc.metadata["type"] == "text":
                outputs = model.get_text_features(**inputs)
            else:
                outputs = model.get_image_features(**inputs)

        embeddings.append(outputs.squeeze().numpy())
    return embeddings

# Get CLIP embeddings for both text and image documents
text_embeddings = get_clip_embeddings(text_documents, clip_model, clip_processor)
image_embeddings = get_clip_embeddings(image_documents, clip_model, clip_processor)

# Combine text and image documents and their embeddings
all_documents = text_documents + image_documents
all_embeddings = np.array(text_embeddings + image_embeddings)

# Create a FAISS index
embedding_dimension = all_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dimension)

# Add embeddings to the index
faiss_index.add(all_embeddings)

# Save the FAISS index
faiss.write_index(faiss_index, "faiss_combined_index.index")

# Load the FAISS index
loaded_index = faiss.read_index("faiss_combined_index.index")

# Verify the number of documents in the combined vector store
num_combined_vectors = loaded_index.ntotal
print(f"Number of documents stored in FAISS vector store: {num_combined_vectors}")

# Ensure the number of vectors matches the number of documents
if num_combined_vectors == len(all_documents):
    print("All documents are successfully stored in the FAISS vector store.")
else:
    print(f"Discrepancy found: {len(all_documents) - num_combined_vectors} documents are missing.")

# Verify the FAISS index type and dimensions for the combined vector store
combined_index_type = type(loaded_index).__name__
combined_dimensionality = loaded_index.d

print(f"Combined FAISS index type: {combined_index_type}")
print(f"Combined FAISS vector dimensionality: {combined_dimensionality}")


Number of documents stored in FAISS vector store: 105
All documents are successfully stored in the FAISS vector store.
Combined FAISS index type: IndexFlatL2
Combined FAISS vector dimensionality: 512


In [ ]:
import os
from PIL import Image
import base64
import io
import numpy as np
from langchain.vectorstores import FAISS
from transformers import CLIPProcessor, CLIPModel, BertModel, BertTokenizer, GPT2Tokenizer
from langchain.schema import Document
import faiss
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Initialize tokenizer for token counting
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Initialize HuggingFace BERT model and tokenizer
bert_model_name = "bert-large-uncased"
bert_model = BertModel.from_pretrained(bert_model_name)
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# Load CLIP model and processor for image embeddings
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define the base directory where the images are stored
image_base_directory = "C:\\Users\\Hp\\Downloads\\images"

def get_clip_embeddings(documents, model, processor):
    embeddings = []
    for doc in documents:
        if doc.metadata["type"] == "text":
            inputs = processor(text=doc.page_content, return_tensors="pt", padding=True, truncation=True)
        else:
            # Load the image from the file path specified in original_content
            image_path = os.path.join(image_base_directory, doc.metadata["original_content"])
            image = Image.open(image_path)
            inputs = processor(images=image, return_tensors="pt")

        with torch.no_grad():
            if doc.metadata["type"] == "text":
                outputs = model.get_text_features(**inputs)
            else:
                outputs = model.get_image_features(**inputs)

        embeddings.append(outputs.squeeze().numpy())
    return embeddings

# Function to retrieve image from metadata
def retrieve_image_from_metadata(metadata):
    image_filename = metadata['original_content']
    image_path = os.path.join(image_base_directory, image_filename)
    image = Image.open(image_path)
    return image

def convert_image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    return img_str

def check_similarity_between_docs(text_docs, image_docs, text_embeddings, image_embeddings):
    # Get embeddings for the top text and image documents
    text_embeddings_list = get_clip_embeddings(text_docs, clip_model, clip_processor)
    image_embeddings_list = get_clip_embeddings(image_docs, clip_model, clip_processor)

    # Compare the first text and image embedding
    if text_embeddings_list and image_embeddings_list:
        text_embedding = np.array(text_embeddings_list[0]).reshape(1, -1)
        image_embedding = np.array(image_embeddings_list[0]).reshape(1, -1)
        similarity_score = cosine_similarity(text_embedding, image_embedding)[0][0]
        print(f"Cosine similarity between the retrieved text and image embeddings: {similarity_score:.4f}")

        if similarity_score > 0.8:  # Adjust the threshold based on your needs
            print("The retrieved text and image embeddings are comparable.")
        else:
            print("The retrieved text and image embeddings are not comparable.")
    else:
        print("No embeddings found for comparison.")

# Function to embed documents using CLIP and BERT models
def embed_documents(documents):
    text_documents = [doc for doc in documents if doc.metadata["type"] == "text"]
    image_documents = [doc for doc in documents if doc.metadata["type"] == "image"]

    text_embeddings = get_clip_embeddings(text_documents, clip_model, clip_processor)
    image_embeddings = get_clip_embeddings(image_documents, clip_model, clip_processor)

    all_embeddings = np.array(text_embeddings + image_embeddings)
    return text_documents + image_documents, all_embeddings

# Load the combined FAISS index
combined_index_path = "faiss_combined_index.index"
loaded_index = faiss.read_index(combined_index_path)

def answer(question):
    # Get the query embedding using BERT
    query_inputs = bert_tokenizer(question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        query_embedding = bert_model(**query_inputs).last_hidden_state.mean(dim=1).numpy()

    # Perform the search in the FAISS index
    distances, indices = loaded_index.search(query_embedding, k=5)

    relevant_documents = [documents[idx] for idx in indices[0]]
    relevant_text_docs = [doc for doc in relevant_documents if doc.metadata["type"] == "text"]
    relevant_image_docs = [doc for doc in relevant_documents if doc.metadata["type"] == "image"]

    # Check similarity between text and image embeddings
    check_similarity_between_docs(relevant_text_docs, relevant_image_docs, clip_model, clip_processor)

    top_contexts = []
    sources = []
    relevant_images = []
    max_tokens = 8192
    completion_tokens = 1024
    current_tokens = 0

    def add_context(context, source_info):
        nonlocal current_tokens
        context_tokens = len(tokenizer.encode(context))
        if current_tokens + context_tokens + completion_tokens <= max_tokens:
            top_contexts.append((len(top_contexts) + 1, context))
            sources.append(source_info)
            current_tokens += context_tokens
            return True
        return False

    for d in relevant_text_docs:
        source_info = f"Source ID: {d.metadata['id']}, Type: {d.metadata['type']}, Source: {d.metadata.get('source', 'Unknown')}"
        context = '[text]' + d.metadata['original_content'] + "\n" + source_info + "\n"
        if not add_context(context, source_info):
            break

    for d in relevant_image_docs:
        source_info = f"Source ID: {d.metadata['id']}, Type: {d.metadata['type']}, Source: {d.metadata['source']}, Page Number: {d.metadata.get('page_number', 'N/A')}"
        image = retrieve_image_from_metadata(d.metadata)
        img_str = convert_image_to_base64(image)
        context = f'[image]<img src="data:image/png;base64,{img_str}" />\n' + source_info + "\n"
        if add_context(context, source_info):
            relevant_images.append(img_str)
        break

    combined_context = "\n\n".join([context for _, context in top_contexts])

    print(f"Combined Context:\n{combined_context}")
    return combined_context, sources, relevant_images, top_contexts

# Example usage:
result, sources, relevant_images, top_contexts = answer("explain the process of fashion cycle")
print("Top contexts:")
for idx, context in top_contexts:
    print(f"Context {idx}:")
    print(context)

print("\nSources:", sources)
print("\nRelevant Images:", relevant_images)
